In [1]:
import pandas as pd
import tensorflow as tf

## 배치 정규화 and/or Batch Normalization
- 배치 정규화는 각 층에서 활성화 값이 적당히 분포되도록 조정하는 것을 목표로 함
- 각 층 사이에 정규화 함수를 사용하여 (가급적이면 활성화 함수 이전에) 정규화 효과를 극대화
- 정규화로 인한 기대 효과는:
> 1. 오버피팅을 억제하고,
> 2. 학습 속도를 향상 시키고
> 3. 학습 효율성도 높임
___

## 데이터 준비

In [2]:
directory = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
boston = pd.read_csv(directory)

In [3]:
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
ind_var = boston.iloc[:,:-1]
dep_var = boston.iloc[:,-1:]

In [5]:
print(ind_var.shape, dep_var.shape)

(506, 13) (506, 1)


## 모델의 구조 만들기

In [11]:
# 3개의 은닉층이 있는 신경망 모델을 다음과 같이 변경
#X = tf.keras.layers.Input([13])
#H = tf.keras.layers.Dense(5, activation='swish')(X) 
#H = tf.keras.layers.Dense(5, activation='swish')(H)
#H = tf.keras.layers.Dense(5, activation='swish')(H)
#y = tf.keras.layers.Dense(1)(H)


X = tf.keras.layers.Input([13])
### 이하 하나의 은닉층 구성
### 기존에는 Dense 함수로 활성화 함수까지 한번에 해결하였으나,
### 확성화 단계를 분리하고 중간에 배치정규화 단계을 삽입
H1 = tf.keras.layers.Dense(5)(X) 
H1 = tf.keras.layers.BatchNormalization()(H1)
H1 = tf.keras.layers.Activation('swish')(H1) 
### 은닉층 2
H2 = tf.keras.layers.Dense(5)(H1) 
H2 = tf.keras.layers.BatchNormalization()(H2)
H2 = tf.keras.layers.Activation('swish')(H2) 
### 은닉층 3
H3 = tf.keras.layers.Dense(5)(H2) 
H3 = tf.keras.layers.BatchNormalization()(H3)
H3 = tf.keras.layers.Activation('swish')(H3) 

y = tf.keras.layers.Dense(1)(H3)

In [12]:
model = tf.keras.models.Model(X, y)
model.compile(loss='mse')

## 모델 학습하기

In [13]:
model.fit(ind_var, dep_var, epochs=1000, verbose=0)
model.fit(ind_var, dep_var, epochs=10)

Epoch 1/10
16/16 [==============================] - 0s 997us/step - loss: 14.8014
Epoch 2/10
16/16 [==============================] - 0s 873us/step - loss: 15.1680
Epoch 3/10
16/16 [==============================] - 0s 811us/step - loss: 13.9301
Epoch 4/10
16/16 [==============================] - 0s 748us/step - loss: 14.0738
Epoch 5/10
16/16 [==============================] - 0s 748us/step - loss: 15.5350
Epoch 6/10
16/16 [==============================] - 0s 686us/step - loss: 13.4166
Epoch 7/10
16/16 [==============================] - 0s 686us/step - loss: 12.0875
Epoch 8/10
16/16 [==============================] - 0s 686us/step - loss: 14.3019
Epoch 9/10
16/16 [==============================] - 0s 686us/step - loss: 14.4109
Epoch 10/10
16/16 [==============================] - 0s 748us/step - loss: 13.8509


## 기타 신경망 모델과 성능 비교

In [14]:
## 단층
X = tf.keras.layers.Input([13])
y = tf.keras.layers.Dense(1)(X)
model2 = tf.keras.models.Model(X, y)
model2.compile(loss='mse')
model2.fit(ind_var, dep_var, epochs=1000, verbose=0)
model2.fit(ind_var, dep_var, epochs=10)

Epoch 1/10
16/16 [==============================] - 0s 623us/step - loss: 27.3344
Epoch 2/10
16/16 [==============================] - 0s 748us/step - loss: 27.6151
Epoch 3/10
16/16 [==============================] - 0s 748us/step - loss: 27.4164
Epoch 4/10
16/16 [==============================] - 0s 748us/step - loss: 27.2049
Epoch 5/10
16/16 [==============================] - 0s 686us/step - loss: 26.9364
Epoch 6/10
16/16 [==============================] - 0s 561us/step - loss: 27.6076
Epoch 7/10
16/16 [==============================] - 0s 561us/step - loss: 27.1145
Epoch 8/10
16/16 [==============================] - 0s 499us/step - loss: 27.4070
Epoch 9/10
16/16 [==============================] - 0s 561us/step - loss: 27.1861
Epoch 10/10
16/16 [==============================] - 0s 624us/step - loss: 27.6177


In [15]:
## 다층 (3)
X = tf.keras.layers.Input([13])
H = tf.keras.layers.Dense(5, activation='swish')(X)
H = tf.keras.layers.Dense(5, activation='swish')(H) 
H = tf.keras.layers.Dense(5, activation='swish')(H) 
y = tf.keras.layers.Dense(1)(H)
model3 = tf.keras.models.Model(X, y)
model3.compile(loss='mse')
model3.fit(ind_var, dep_var, epochs=1000, verbose=0)
model3.fit(ind_var, dep_var, epochs=10)

Epoch 1/10
16/16 [==============================] - 0s 686us/step - loss: 17.2474
Epoch 2/10
16/16 [==============================] - 0s 748us/step - loss: 16.7583
Epoch 3/10
16/16 [==============================] - 0s 810us/step - loss: 17.4714
Epoch 4/10
16/16 [==============================] - 0s 810us/step - loss: 17.0718
Epoch 5/10
16/16 [==============================] - 0s 748us/step - loss: 16.6776
Epoch 6/10
16/16 [==============================] - 0s 623us/step - loss: 17.2957
Epoch 7/10
16/16 [==============================] - 0s 685us/step - loss: 16.8425
Epoch 8/10
16/16 [==============================] - 0s 623us/step - loss: 17.1420
Epoch 9/10
16/16 [==============================] - 0s 499us/step - loss: 16.5484
Epoch 10/10
16/16 [==============================] - 0s 623us/step - loss: 16.9835


- 각각 loss를 살펴보면 근소하지만, 정규화 모델이 우수한 결과를 보여주는 것을 확인 가능
> 1. 단층: 27.6
> 2. 다층(3): 17.0
> 3. 다층 정규화(3): 13.9